<a href="https://colab.research.google.com/github/VietHoangg03/AppMobile_BN/blob/main/ChatBot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jul  1 08:12:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# https://drive.google.com/file/d/1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx/view?usp=drive_link
!gdown 1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx

Downloading...
From: https://drive.google.com/uc?id=1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx
To: /content/YOLOv10_Tutorials.pdf
100% 16.6M/16.6M [00:00<00:00, 75.7MB/s]


In [ ]:
!pip install -q transformers==4.52.4
!pip install -q bitsandbytes==0.46.0
!pip install -q accelerate==1.7.0
!pip install -q langchain==0.3.25
!pip install -q langchainhub==0.1.21
!pip install -q langchain-chroma==0.2.4
!pip install -q langchain_experimental==0.3.4
!pip install -q langchain-community==0.3.24
!pip install -q langchain_huggingface==0.2.0
!pip install -q python-dotenv==1.1.0
!pip install -q pypdf
!pip install -U transformers

In [ ]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import SemanticChunker

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [ ]:
Loader = PyPDFLoader
FILE_PATH = "./YOLOv10_Tutorials.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

print("Number of documents: ", len(documents))
documents[0]

Number of documents:  20


Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-06-20T11:06:10+00:00', 'author': '', 'keywords': '', 'moddate': '2024-06-20T11:06:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Overleaf Example', 'trapped': '/False', 'source': './YOLOv10_Tutorials.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng)là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương 

In [ ]:
print(documents[0])
print(documents[1])

page_content='AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng)là một bài toán cổ điển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look
Once) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)
đã đề xuất mô hìnhYOLOv10 trong bài báoYOLOv10: Real-Time End-to-End Object
Detection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu suất vượt trội hơn so với các
phiên bản 

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1,                          # Nhóm 3 câu
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    # number_of_chunks= 30,                   # Số chunks mong muốn. Tìm number_of_chunks - 1 điểm có similarity thấp nhất để cắt. Nếu không đủ điểm cắt phù hợp → số chunks thực tế ≠ số chunks mong muốn
    min_chunk_size=500,                     # Tối thiểu 1000 ký tự
    # sentence_split_regex=r'(?<=[.?!…])\s+', # Bao gồm dấu … cho tiếng Việt
    add_start_index=True
)

In [ ]:
docs = semantic_splitter.split_documents(documents)
print("Number of semantic chunks: ", len(docs))

Number of semantic chunks:  32


In [ ]:
docs[6]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-06-20T11:06:10+00:00', 'author': '', 'keywords': '', 'moddate': '2024-06-20T11:06:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Overleaf Example', 'trapped': '/False', 'source': './YOLOv10_Tutorials.pdf', 'total_pages': 20, 'page': 4, 'page_label': '5', 'start_index': 0}, page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn\n- Điểm mới: Sử dụng anchor boxes, mạng Darknet-19, và tăng training data để tăng độ\nchính xác. - Ưu điểm: Tăng độ chính xác và khả năng nhận diện nhiều object trong 1 cell. - Nhược điểm: Phức tạp hơn, cần nhiều tài nguyên tính toán, và khó detect các object\nnhỏ. II.IV. YOLOv3\nYOLOv3 [3] ra mắt năm 2018, tiếp tục cải tiến từ YOLOv2. Hình 6: Kiến trúc mô hình YOLOv3. Ảnh: [3]. - Điểm mới: Sử dụng mạng Darknet-53 và detect object ở ba cấp độ khác nhau (multi-scale\n

In [ ]:
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings)

retriever = vector_db.as_retriever()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
querry = "Yolov10 dùng để làm gì?"
result = retriever.invoke(querry) #Chuyển thành 1 vector truy vấn

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
len(result)

4

In [ ]:
result[0]

Document(id='d09ba737-6b5a-4841-a736-7196546a181a', metadata={'page_label': '2', 'creationdate': '2024-06-20T11:06:10+00:00', 'total_pages': 20, 'title': 'Overleaf Example', 'author': '', 'creator': 'LaTeX with hyperref', 'keywords': '', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': './YOLOv10_Tutorials.pdf', 'page': 1, 'moddate': '2024-06-20T11:06:10+00:00', 'subject': '', 'trapped': '/False', 'start_index': 0}, page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn\nHình 2: Hiệu suất của YOLOv10 khi so sánh với các mô hình khác. Trên tập dữ liệu COCO,\nYOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô\nhình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao. Ảnh: [10]. Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu về YOLOv10 và cách sử dụng mô hình này. Thông qua đó, nhóm cũng sẽ trình bày sơ lược về bài toán Object Detect

In [ ]:
result[1]

Document(id='68780a50-6704-4b87-b3fc-575d176523c8', metadata={'source': './YOLOv10_Tutorials.pdf', 'page_label': '9', 'total_pages': 20, 'moddate': '2024-06-20T11:06:10+00:00', 'keywords': '', 'page': 8, 'trapped': '/False', 'subject': '', 'creationdate': '2024-06-20T11:06:10+00:00', 'creator': 'LaTeX with hyperref', 'title': 'Overleaf Example', 'author': '', 'producer': 'pdfTeX-1.40.25', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'start_index': 0}, page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn\nIII. YOLOv10: Real-Time End-to-End\nObject Detection\nAo Wang và các cộng sự đã đặt nghi vấn về sự tối ưu trong việc phụ thuộc vào kỹ thuật hậu xử\nlý Non-maximum Suppresion (NMS) và cách thiết kế mô hình của các phiên bản YOLO trước\nđó. Với các hạn chế quan sát được từ hai điều trên và mục tiêu xây dựng một mô hình object\ndetection thời gian thực, YOLOv10 đã được đề xuất với những thay đổi mới. Theo đó, có hai\nđiểm nhấn

In [ ]:
# Load LLMs
!pip install -U bitsandbytes
!pip install -U transformers bitsandbytes



In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
USER_QUESTION = "YOLOv10 là gì?"

# Invoke the chain
output = rag_chain.invoke(USER_QUESTION)
print(output)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: YOLOv10 là gì? 
Context: YOLOv9
YOLOv9 [9] được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark
Liao. Mô hình này cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật
mới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation
Network (GELAN). 7

AI VIETNAM (AIO2024) aivietnam.edu.vn
Hình 2: Hiệu suất của YOLOv10 khi so sánh với các mô hình khác. Trên tập dữ liệu COCO,
YOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô
hình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao. Ảnh: [10]. Trong bài viết này, chúng ta sẽ cùng nhau tìm hiểu về YOLOv10 và cách sử dụng mô hình này. Thông qua đó, nhóm cũng sẽ

In [ ]:
answer = output.split('Answer:')[1].strip()
answer

'YOLOv10 là một phiên bản mới của YOLO, được giới thiệu vào năm 2024. Nó cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật mới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation Network (GELAN). Khi so sánh với các mô hình khác, YOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô hình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao.'

In [ ]:
!pip install -q streamlit==1.46.0